In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing # for label encoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [29]:
data_orig = pd.read_csv("data\play_by_play_2019.csv", dtype=object) # maybe should be dtype=str
# Trim up our data to only features we care about
feature_vectors = data_orig[["touchdown", "home_team", "away_team", "season_type", "yardline_100", "quarter_seconds_remaining", "half_seconds_remaining",
                 "game_seconds_remaining", "game_half", "drive", "sp", "qtr", "down", "goal_to_go", "ydstogo", "play_type", "rush_attempt", "pass_attempt" ]]
labels = data_orig[["touchdown"]] # 1 or 0
# Once I get to a certain point, I should try adding data from other csv files to this dataframe as well

In [22]:
# In this cell I should add ELO ratings to each row in the data.  
# Going to skip that for now and just do this with data we already have

In [27]:
# Encode columns in feature_vectors so that there aren't any strings
# Columns we need to encode: home_team, away_team, season_type, game_half, down (convert NA to 0), play_type, 
# rush_attempt (NAs), pass_attempt (NAs), 
le = preprocessing.LabelEncoder()
feature_vectors["home_team"] = le.fit_transform(feature_vectors["home_team"])
feature_vectors["away_team"] = le.fit_transform(feature_vectors["away_team"])
feature_vectors["season_type"] = le.fit_transform(feature_vectors["season_type"])
feature_vectors["game_half"] = le.fit_transform(feature_vectors["game_half"])

# Go through and fill in missing values for the columns that initially had NA values
feature_vectors["down"] = feature_vectors["down"].fillna(-1)
feature_vectors["play_type"] = feature_vectors["play_type"].fillna(-1)
feature_vectors["rush_attempt"] = feature_vectors["rush_attempt"].fillna(-1)
feature_vectors["pass_attempt"] = feature_vectors["pass_attempt"].fillna(-1)


shape of feature vectors:  (48034, 17)
shape of training data:  (38427, 17)
type of training data:  <class 'pandas.core.frame.DataFrame'>

shape of test data:  (9607, 17)
type of test data:  <class 'pandas.core.frame.DataFrame'>


<ipython-input-27-25c063992bc2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_vectors["home_team"] = le.fit_transform(feature_vectors["home_team"])
<ipython-input-27-25c063992bc2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_vectors["away_team"] = le.fit_transform(feature_vectors["away_team"])
<ipython-input-27-25c063992bc2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [34]:
# Get the training and test data
train_data, test_data = train_test_split(feature_vectors, test_size=0.2, random_state=35, shuffle=True)

train_labels = train_data[["touchdown"]]
test_labels = test_data[["touchdown"]]

train_data = train_data.drop("touchdown", 1)
test_data = test_data.drop("touchdown", 1)

In [36]:
# Set up our KNN model

model = KNeighborsClassifier(n_neighbors=5)

#feature_vectors to numpy array yields #rows = # data points and columns = dim feature vectors (# features)
# labels is # datapoints x 1
#model.fit(feature_vectors.to_numpy(), labels.to_numpy()) #fit(features, labels) --> arguments should be array like
model.fit(train_data, train_labels)
print("test data 0: ", test_data[0])
print("labels 0: ", test_labels[0])
#predicted = model.predict(test_data[0])
#print("Predicted: ", predicted)
#print("Actual Label: ", test_labels[0])

ValueError: could not convert string to float: 'ARI'